In [11]:
from transformers import MusicgenForConditionalGeneration

model = MusicgenForConditionalGeneration.from_pretrained("facebook/musicgen-small")

/Users/joanneliu/Desktop/AI4DM/H2M1/FlaskApp_Template/.venv/lib/python3.9/site-packages/transformers/models/encodec/modeling_encodec.py:124: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.register_buffer("padding_total", torch.tensor(kernel_size - stride, dtype=torch.int64), persistent=False)


In [12]:
from openai import OpenAI

In [13]:
import torch

device = "cuda:0" if torch.cuda.is_available() else "cpu"
model.to(device);

In [14]:
# Use a pipeline as a high-level helper
from transformers import pipeline
from dotenv import load_dotenv


In [15]:
load_dotenv()

True

In [16]:
from IPython.display import Audio
from transformers import AutoProcessor


10.24

In [26]:
def poemGenerator(msg):

  client = OpenAI()

  msg_list = [
      {"role": "system", "content": "You are a expert poem writer. Using deep meaning and short lines or words you generate poem in less than 20 words based on the given emotion."},
    ]

  msg_list.append(msg)
  response = client.chat.completions.create(
    model="gpt-4o-mini",
    temperature=0.2,
    max_completion_tokens=100,
    messages = msg_list
  )

  out_message = response.choices[0].message.content
  return out_message

In [27]:
msg = {"role": "user", "content": "i am feeling a bit sad but also nervous at the same time"}
poem = poemGenerator(msg)
print(poem)

Whispers of doubt,  
shadows of sorrow,  
hearts heavy,  
breath quickens,  
hope flickers,  
tomorrow waits.


In [24]:
audio_length_in_s = 512 / model.config.audio_encoder.frame_rate
sampling_rate = model.config.audio_encoder.sampling_rate

audio_length_in_s
processor = AutoProcessor.from_pretrained("facebook/musicgen-small")

inputs = processor(
    text=["background music with light melody for the poem" + out_message],
    padding=True,
    return_tensors="pt",
)

audio_values = model.generate(**inputs.to(device), do_sample=True, guidance_scale=3, max_new_tokens=512)
Audio(audio_values[0].cpu().numpy(), rate=sampling_rate)
